<h1> EY Job Scraper </h1>
 
Find each country's EY job posting information on https://ey.jobs/ 

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import random
from random import randint

In [2]:
states = ['California', 'New-York','Texas','Georgia','Illinois','Virginia','Massachusetts','New-Jersey',
          'North-Carolina', 'Ohio','Florida', 'Pennsylvania', 'Washington', 'Tennessee', 'Minnesota','Connecticut',
          'Michigan','Colorado','District-Of-Columbia','Maryland','Missouri','Oklahoma','Oregon','Utah','Kentucky',
          'Louisiana','Nevada','Arizona','South-Carolina','Arkansas','Hawaii','Indiana','Rhode-Island','Wisconsin',
          'Iowa']

state_links = dict()
for state in states:
    state_links[state] = 'https://ey.jobs/'+state+'/usa/jobs'

In [4]:
info = dict()
info['job_link'] = list()
info['region'] = list()
info['country'] = list()

In [5]:
# for the US
req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }

with requests.Session() as s:
    for state in states:
        r = s.get(state_links[state],headers = req_headers)
        soup = BeautifulSoup(r.content, 'lxml')
        job_tags = soup.find_all('li',{'class':'direct_joblisting'})
        for i in range(len(job_tags)):
            try:
                info['job_link'].append(job_tags[i].find('a').get('href'))
                info['region'].append(state)
                info['country'].append('United States')
            except:
                continue
            time.sleep(random.randint(2,5))
        time.sleep(random.randint(6,10))

In [6]:
df = pd.DataFrame(info)
df.head()

,job_link,region,country
0,/san-francisco-ca/wavespace-facilitator-strate...,California,United States
1,/los-angeles-ca/manager-restructuring-nyc-chic...,California,United States
2,/san-francisco-ca/tax-services-senior-national...,California,United States
3,/los-angeles-ca/people-advisory-services-workf...,California,United States
4,/los-angeles-ca/assurance-services-experienced...,California,United States


In [76]:
info_2 = dict()
info_2['job_description'] = list()
info_2['job_title'] = list()
info_2['job_link'] = list()

In [153]:
def scrape_info(link):
    url = 'https://ey.jobs'+link
    try:
        
        with requests.Session() as s:    
            r = s.get(url,headers = req_headers)
            soup = BeautifulSoup(r.content, 'lxml')
            title = soup.find('span',{'class':"direct_highlightedText"}).find('span',{'itemprop':"title"}).get_text()
            l = soup.find('div',{'itemprop':"description"}).find_all('p')
            job_description = ''
            for i in l:
                job_description += i.get_text()
        return (title,job_description,link)
    except:
         pass # some job might no longer exist and need to be removed 

  

In [158]:
for i in range(len(info['job_link'])):
    try:
        l = info['job_link'][i]
        t = scrape_info(l)
        info_2['job_description'].append(t[1])
        info_2['job_title'].append(t[0])
        info_2['job_link'].append(t[2])
        time.sleep(random.randint(2,5))
    except:
        continue
    

In [160]:
df = pd.DataFrame(info)

In [161]:
df_2 = pd.DataFrame(info_2)

In [162]:
df = df.drop_duplicates()
df_2 = df_2.drop_duplicates()

In [163]:
df_m = pd.merge(df, df_2, on='job_link', how='right')

In [164]:
df_m

,job_link,region,country,job_description,job_title
0,/san-francisco-ca/wavespace-facilitator-strate...,California,United States,Wavespace Facilitator (Strategy) - Financial S...,Wavespace Facilitator (Strategy) - Financial S...
1,/los-angeles-ca/manager-restructuring-nyc-chic...,California,United States,"Manager - Restructuring- NYC, Chicago, Dallas ...","Manager - Restructuring- NYC, Chicago, Dallas ..."
2,/san-francisco-ca/tax-services-senior-national...,California,United States,Tax Services Senior - National Tax - Transfer ...,Tax Services Senior - National Tax - Transfer ...
3,/los-angeles-ca/people-advisory-services-workf...,California,United States,People Advisory Services - Workforce Advisory/...,People Advisory Services - Workforce Advisory/...
4,/los-angeles-ca/assurance-services-experienced...,California,United States,Assurance Services Experienced Staff - Externa...,Assurance Services Experienced Staff - Externa...
5,/san-francisco-ca/restructuring-senior-nyc-chi...,California,United States,"Restructuring Senior- NYC, Chicago, Dallas, Ho...","Restructuring Senior- NYC, Chicago, Dallas, Ho..."
6,/los-angeles-ca/people-advisory-services-workf...,California,United States,People Advisory Services - Workforce Advisory/...,People Advisory Services - Workforce Advisory/...
7,/los-angeles-ca/restructuring-senior-nyc-chica...,California,United States,"Restructuring Senior- NYC, Chicago, Dallas, Ho...","Restructuring Senior- NYC, Chicago, Dallas, Ho..."
8,/san-diego-ca/assurance-services-senior-extern...,California,United States,Assurance Services Senior - External AuditAssu...,Assurance Services Senior - External Audit
9,/los-angeles-ca/people-advisory-services-workf...,California,United States,People Advisory Services - Workforce Advisory/...,People Advisory Services - Workforce Advisory/...


In [165]:
len(df_m)

654

In [8]:
df.to_csv('links.csv')

In [173]:
df_m.to_csv('job_description_usa.csv')

In [36]:
req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }
    
with requests.Session() as s:
    page_url = 'http://ey.jobs'
    r = s.get(page_url, headers=req_headers)
    soup = BeautifulSoup(r.content, 'lxml')
    country = soup.find('ul', {'role': 'menu'}).find_all()    
    

In [37]:
country_dict = dict()
for i in range(len(country)):
    try:
        country_dict[country[i].get_text()] = country[i].find('a').get('href')
    except:
        country_dict[country[i].get_text()] = country[i].get('href')

In [39]:
del country_dict['United States (1,478)'] # we have already had this 

In [44]:
country_dict_2 = dict()
import re
for key in country_dict.keys():
    pattern = re.compile(r'^([a-zA-Z ]+)(\s\D{1}[0-9]+\D{1})$')
    match = re.search(pattern, key)
    new_key = match.groups()[0]
    country_dict_2[new_key] = country_dict[key]

In [170]:
country_dict = country_dict_2

In [192]:
info_other = dict()
info_other['job_link'] = list()
info_other['region'] = list()
info_other['country'] = list()

In [194]:
# for other countries
with requests.Session() as s:
    for country in country_dict.keys():
        r = s.get('https://ey.jobs'+country_dict[country],headers = req_headers)
        soup = BeautifulSoup(r.content, 'lxml')
        job_tags = soup.find_all('li',{'class':'direct_joblisting'})
        for i in range(len(job_tags)):
            try:
                info_other['job_link'].append(job_tags[i].find('a').get('href'))
                region = job_tags[i].find('span',{'class': "hiringPlace"}).get_text().strip().replace('\n','').split(',')[0]
                info_other['region'].append(region)
                info_other['country'].append(country)
            except:
                continue
            time.sleep(random.randint(2,5))
        time.sleep(random.randint(6,10))

KeyboardInterrupt: 

In [178]:
with requests.Session() as s:
    r = s.get('https://ey.jobs'+country_dict['United Kingdom'],headers = req_headers)
    soup = BeautifulSoup(r.content, 'lxml')
    job_tags = soup.find_all('li',{'class':'direct_joblisting'})

In [204]:
set(info_other['country'])

{'Argentina',
 'Australia',
 'Belgium',
 'Canada',
 'China',
 'India',
 'Ireland',
 'Luxembourg',
 'Philippines',
 'Poland',
 'Singapore',
 'South Africa',
 'Spain',
 'Switzerland',
 'United Kingdom'}

In [203]:
list(country_dict.keys())

['United Kingdom',
 'India',
 'Canada',
 'Poland',
 'Australia',
 'Belgium',
 'Singapore',
 'China',
 'Argentina',
 'Philippines',
 'Luxembourg',
 'South Africa',
 'Spain',
 'Ireland',
 'Switzerland',
 'Saudi Arabia',
 'New Zealand',
 'Greece',
 'Indonesia']

In [ ]:
country[1].get_text()

In [ ]:
country[1]

In [ ]:
req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.52 Safari/536.5'
}
with requests.Session() as s:
    page_url = 'http://ey.jobs/usa/jobs/#200'
    
    r = s.get(page_url,verify = False)
    soup = BeautifulSoup(r.content, 'lxml')

In [ ]:
len(soup.find_all('li',{'class':"direct_joblisting"}))

In [ ]:
soup

In [ ]:
r = requests.get('http://ey.jobs/usa/jobs/#3',timeout=5)

In [ ]:
for html in r.html:
    print(html)

In [ ]:
soup = BeautifulSoup(r.content, 'lxml')